In [35]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [36]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [37]:
tf.keras.backend.clear_session()

In [38]:
subject = 'Mato Grosso - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [39]:
data = pd.read_csv('2003_mo_model_input_MT.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Mato Grosso - Produção de Cimento (t),Mato Grosso - IDH Longevidade,Mato Grosso - IDH Educacao,Mato Grosso - Desemprego,Mato Grosso - IDH Renda,Mato Grosso - IDH,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,...,Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Mato Grosso - Consumo de Cimento (t)
0,2003-1,50.917240,0.774167,0.713365,8.297178,0.756038,0.745147,6.084564e+07,2.670588e+06,16.831332,...,1.921789e+10,8.202881e+10,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,47.470
1,2003-2,50.851586,0.774487,0.713721,8.291317,0.755860,0.745308,6.092768e+07,2.672688e+06,16.835967,...,1.921789e+10,8.222716e+10,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,45.387
2,2003-3,50.970549,0.774808,0.714077,8.285457,0.755682,0.745469,6.100973e+07,2.674788e+06,16.840602,...,1.921789e+10,8.234287e+10,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,44.907
3,2003-4,51.049978,0.775128,0.714434,8.279596,0.755503,0.745630,6.109177e+07,2.676888e+06,16.845237,...,1.921789e+10,8.235526e+10,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,45.467
4,2003-5,51.567529,0.775449,0.714790,8.273735,0.755325,0.745791,6.117381e+07,2.678988e+06,16.849872,...,1.921843e+10,8.245980e+10,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,56.246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,2022-9,118.223448,NaN,NaN,NaN,0.732410,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,183.895
237,2022-10,117.524152,NaN,NaN,NaN,0.732498,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,180.525
238,2022-11,116.430559,NaN,NaN,NaN,0.732587,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,161.683
239,2022-12,116.846011,NaN,NaN,NaN,0.732676,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,161.683


In [40]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Mato Grosso - Produção de Cimento (t),Mato Grosso - IDH Longevidade,Mato Grosso - IDH Educacao,Mato Grosso - Desemprego,Mato Grosso - IDH Renda,Mato Grosso - IDH,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,...,Títulos – LIG (R$),Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%)
0,-1.582351,-2.923645,-2.002719,1.216878,1.933600,-2.486031,-1.594683,-2.079312,-1.524159,-1.566766,...,1.922051,-1.023713,-0.767541,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582
1,-1.585655,-2.833952,-1.975252,1.204107,1.908621,-2.439296,-1.579788,-2.049593,-1.511861,-1.551683,...,1.922051,-1.023713,-0.762847,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356
2,-1.579669,-2.744258,-1.947785,1.191337,1.883642,-2.392562,-1.564893,-2.019873,-1.499562,-1.536601,...,1.922051,-1.023713,-0.760108,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895
3,-1.575671,-2.654564,-1.920318,1.178566,1.858663,-2.345827,-1.549998,-1.990154,-1.487263,-1.521519,...,1.922051,-1.023713,-0.759815,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628
4,-1.549625,-2.564870,-1.892851,1.165795,1.833684,-2.299093,-1.535104,-1.960435,-1.474964,-1.506437,...,1.916280,-1.023685,-0.757341,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.683341,-1.379747,1.247049,-1.462024,-1.234751,0.554708,1.263263,0.134707,1.180043,1.228996,...,1.977516,1.711283,1.052051,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976
188,0.683776,-1.476985,1.242347,-1.475003,-1.234325,0.507932,1.258638,0.140184,1.177055,1.225355,...,2.193251,1.722665,1.023064,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005
189,0.711014,-1.574222,1.237645,-1.487981,-1.233899,0.461157,1.254013,0.145662,1.174067,1.221714,...,2.413615,1.735332,0.994034,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489
190,0.715019,-1.671459,1.232943,-1.500960,-1.233473,0.414381,1.249388,0.151139,1.171079,1.218073,...,2.629916,1.748418,0.966056,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492


In [41]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      51.972
1      40.729
2      54.446
3      51.788
4      58.702
        ...  
236       NaN
237       NaN
238       NaN
239       NaN
240       NaN
Name: Mato Grosso - Consumo de Cimento (t), Length: 241, dtype: float64

In [42]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Mato Grosso - Produção de Cimento (t),Mato Grosso - IDH Longevidade,Mato Grosso - IDH Educacao,Mato Grosso - Desemprego,Mato Grosso - IDH Renda,Mato Grosso - IDH,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,...,Títulos – LIG (R$),Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%)
0,-1.582351,-2.923645,-2.002719,1.216878,1.933600,-2.486031,-1.594683,-2.079312,-1.524159,-1.566766,...,1.922051,-1.023713,-0.767541,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582
1,-1.585655,-2.833952,-1.975252,1.204107,1.908621,-2.439296,-1.579788,-2.049593,-1.511861,-1.551683,...,1.922051,-1.023713,-0.762847,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356
2,-1.579669,-2.744258,-1.947785,1.191337,1.883642,-2.392562,-1.564893,-2.019873,-1.499562,-1.536601,...,1.922051,-1.023713,-0.760108,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895
3,-1.575671,-2.654564,-1.920318,1.178566,1.858663,-2.345827,-1.549998,-1.990154,-1.487263,-1.521519,...,1.922051,-1.023713,-0.759815,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628
4,-1.549625,-2.564870,-1.892851,1.165795,1.833684,-2.299093,-1.535104,-1.960435,-1.474964,-1.506437,...,1.916280,-1.023685,-0.757341,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,0.341278,1.012212,1.345393,-0.104237,-1.375426,1.450474,1.217726,0.274296,1.163104,1.185879,...,-0.689873,1.328987,1.557720,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659
158,0.322813,0.945400,1.340059,-0.191347,-1.371825,1.427979,1.221923,0.260732,1.162467,1.188073,...,-0.689873,1.355196,1.561577,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771
159,0.303209,0.878587,1.334725,-0.278458,-1.368224,1.405484,1.226120,0.247169,1.161831,1.190267,...,-0.689873,1.381167,1.561138,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465
160,0.275768,0.811775,1.329391,-0.365569,-1.364623,1.382988,1.230317,0.233605,1.161195,1.192461,...,-0.689873,1.408923,1.559643,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668


In [43]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0       51.972
1       40.729
2       54.446
3       51.788
4       58.702
        ...   
157     65.293
158     90.338
159     83.188
160    107.230
161    105.397
Name: Mato Grosso - Consumo de Cimento (t), Length: 162, dtype: float64

In [44]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [45]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 47)

In [46]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Mato Grosso - Produção de Cimento (t),Mato Grosso - IDH Longevidade,Mato Grosso - IDH Educacao,Mato Grosso - Desemprego,Mato Grosso - IDH Renda,Mato Grosso - IDH,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,...,Títulos – LIG (R$),Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%)
126,1.688924,0.192562,0.620782,-0.994909,-0.680367,0.676307,0.761555,1.240620,0.847032,0.811600,...,-0.689873,0.403484,0.557912,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243
127,1.661012,0.152102,0.628695,-0.917460,-0.675342,0.672501,0.782226,1.224112,0.864923,0.830440,...,-0.689873,0.436846,0.616673,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828
128,1.610628,0.111643,0.636609,-0.840010,-0.670318,0.668694,0.802896,1.207603,0.882814,0.849281,...,-0.689873,0.470723,0.676845,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225
129,1.567145,0.071183,0.644522,-0.762561,-0.665293,0.664888,0.823566,1.191095,0.900705,0.868121,...,-0.689873,0.505118,0.738444,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522
130,1.512872,0.030723,0.652435,-0.685111,-0.660269,0.661082,0.844236,1.174587,0.918596,0.886961,...,-0.689873,0.540037,0.801485,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.683341,-1.379747,1.247049,-1.462024,-1.234751,0.554708,1.263263,0.134707,1.180043,1.228996,...,1.977516,1.711283,1.052051,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976
188,0.683776,-1.476985,1.242347,-1.475003,-1.234325,0.507932,1.258638,0.140184,1.177055,1.225355,...,2.193251,1.722665,1.023064,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005
189,0.711014,-1.574222,1.237645,-1.487981,-1.233899,0.461157,1.254013,0.145662,1.174067,1.221714,...,2.413615,1.735332,0.994034,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489
190,0.715019,-1.671459,1.232943,-1.500960,-1.233473,0.414381,1.249388,0.151139,1.171079,1.218073,...,2.629916,1.748418,0.966056,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492


In [47]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 47)

In [48]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [49]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(180, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(108, return_sequences=True, activation='tanh'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.LSTM(36, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [50]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [51]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[4183141188, 4029364095, 2957117904, 675716824, 1003597351, 1947010775, 1799064885, 2416493150, 2550879253, 3853302514, 3600805840, 1227584282, 1187035836, 1726901934, 1051819394, 3597492344, 2956817011, 1639494963, 301643899, 2252255533, 3972024643, 2815144312, 2597157259, 1113922132, 2806454896, 1701866894, 800675296, 1070500323, 500767085, 1143207522, 760573724, 3176745355, 1604059527, 641041072, 1467032490, 1095736648, 2366617460, 807719796, 3769584735, 1219853389, 4029098928, 1738599272, 3166779793, 3810934411, 1116162600, 2509830643, 2910241909, 1425441421, 1361577618, 593333874]


Step: 0 ___________________________________________
val_loss: 107.0946273803711
winner_seed: 4183141188


Step: 1 ___________________________________________
val_loss: 102.40113830566406
winner_seed: 4029364095


Step: 2 ___________________________________________
val_loss: 104.15034484863281


Step: 3 ___________________________________________
val_loss: 90.41645050048828
winner_seed: 675716824


Step

In [52]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 3s 166ms/step - loss: 8023.0957 - val_loss: 4144.8403
Epoch 2/10000
4/4 [==============================] - 0s 14ms/step - loss: 2679.3333 - val_loss: 539.5164
Epoch 3/10000
4/4 [==============================] - 0s 13ms/step - loss: 535.3296 - val_loss: 449.4268
Epoch 4/10000
4/4 [==============================] - 0s 14ms/step - loss: 498.9250 - val_loss: 448.4494
Epoch 5/10000
4/4 [==============================] - 0s 12ms/step - loss: 507.5028 - val_loss: 451.8680
Epoch 6/10000
4/4 [==============================] - 0s 14ms/step - loss: 498.7730 - val_loss: 448.3686
Epoch 7/10000
4/4 [==============================] - 0s 13ms/step - loss: 491.6800 - val_loss: 447.9794
Epoch 8/10000
4/4 [==============================] - 0s 13ms/step - loss: 500.8838 - val_loss: 439.8989
Epoch 9/10000
4/4 [==============================] - 0s 12ms/step - loss: 490.8812 - val_loss: 431.2730
Epoch 10/10000
4/4 [==============================] - 0s 12m

4/4 [==============================] - 0s 12ms/step - loss: 153.1469 - val_loss: 146.5887
Epoch 80/10000
4/4 [==============================] - 0s 13ms/step - loss: 173.9017 - val_loss: 237.3075
Epoch 81/10000
4/4 [==============================] - 0s 13ms/step - loss: 183.8078 - val_loss: 193.4216
Epoch 82/10000
4/4 [==============================] - 0s 12ms/step - loss: 188.5260 - val_loss: 228.1952
Epoch 83/10000
4/4 [==============================] - 0s 12ms/step - loss: 136.7285 - val_loss: 196.5882
Epoch 84/10000
4/4 [==============================] - 0s 12ms/step - loss: 150.2960 - val_loss: 201.3591
Epoch 85/10000
4/4 [==============================] - 0s 13ms/step - loss: 179.3940 - val_loss: 231.0641
Epoch 86/10000
4/4 [==============================] - 0s 12ms/step - loss: 217.7861 - val_loss: 194.9243
Epoch 87/10000
4/4 [==============================] - 0s 12ms/step - loss: 158.1730 - val_loss: 170.7760
Epoch 88/10000
4/4 [==============================] - 0s 12ms/step - l

4/4 [==============================] - 0s 13ms/step - loss: 136.7375 - val_loss: 232.0888
Epoch 157/10000
4/4 [==============================] - 0s 12ms/step - loss: 114.7940 - val_loss: 254.1408
Epoch 158/10000
4/4 [==============================] - 0s 12ms/step - loss: 117.4748 - val_loss: 254.3643
Epoch 159/10000
4/4 [==============================] - 0s 12ms/step - loss: 110.3506 - val_loss: 188.8802
Epoch 160/10000
4/4 [==============================] - 0s 12ms/step - loss: 119.7667 - val_loss: 265.4471
Epoch 161/10000
4/4 [==============================] - 0s 13ms/step - loss: 105.4353 - val_loss: 261.9672
Epoch 162/10000
4/4 [==============================] - 0s 13ms/step - loss: 103.3548 - val_loss: 316.2668
Epoch 163/10000
4/4 [==============================] - 0s 12ms/step - loss: 114.5615 - val_loss: 260.1786
Epoch 164/10000
4/4 [==============================] - 0s 12ms/step - loss: 108.6605 - val_loss: 267.7872
Epoch 165/10000
4/4 [==============================] - 0s 12ms

4/4 [==============================] - 0s 13ms/step - loss: 103.8485 - val_loss: 248.1912
Epoch 234/10000
4/4 [==============================] - 0s 12ms/step - loss: 105.4466 - val_loss: 255.1781
Epoch 235/10000
4/4 [==============================] - 0s 12ms/step - loss: 101.7973 - val_loss: 255.8138
Epoch 236/10000
4/4 [==============================] - 0s 12ms/step - loss: 102.1214 - val_loss: 275.4893
Epoch 237/10000
4/4 [==============================] - 0s 12ms/step - loss: 92.7689 - val_loss: 252.6134
Epoch 238/10000
4/4 [==============================] - 0s 12ms/step - loss: 99.9847 - val_loss: 272.0030
Epoch 239/10000
4/4 [==============================] - 0s 12ms/step - loss: 95.4793 - val_loss: 254.3735
Epoch 240/10000
4/4 [==============================] - 0s 13ms/step - loss: 101.8906 - val_loss: 259.0612
Epoch 241/10000
4/4 [==============================] - 0s 13ms/step - loss: 101.4471 - val_loss: 292.5502
Epoch 242/10000
4/4 [==============================] - 0s 13ms/st

4/4 [==============================] - 0s 12ms/step - loss: 110.6107 - val_loss: 218.8468
Epoch 311/10000
4/4 [==============================] - 0s 12ms/step - loss: 111.5756 - val_loss: 203.0607
Epoch 312/10000
4/4 [==============================] - 0s 12ms/step - loss: 101.3924 - val_loss: 221.5648
Epoch 313/10000
4/4 [==============================] - 0s 12ms/step - loss: 92.6326 - val_loss: 209.6021
Epoch 314/10000
4/4 [==============================] - 0s 12ms/step - loss: 101.4341 - val_loss: 208.5132
Epoch 315/10000
4/4 [==============================] - 0s 12ms/step - loss: 104.2549 - val_loss: 224.0722
Epoch 316/10000
4/4 [==============================] - 0s 12ms/step - loss: 97.3256 - val_loss: 222.2443
Epoch 317/10000
4/4 [==============================] - 0s 12ms/step - loss: 105.1193 - val_loss: 235.8380
Epoch 318/10000
4/4 [==============================] - 0s 12ms/step - loss: 109.5759 - val_loss: 193.4892
Epoch 319/10000
4/4 [==============================] - 0s 13ms/s

...

4/4 [==============================] - 0s 11ms/step - loss: 58.5918 - val_loss: 139.7594
Epoch 1784/10000
4/4 [==============================] - 0s 11ms/step - loss: 30.8564 - val_loss: 130.3431
Epoch 1785/10000
4/4 [==============================] - 0s 12ms/step - loss: 39.9533 - val_loss: 115.6336
Epoch 1786/10000
4/4 [==============================] - 0s 12ms/step - loss: 40.3919 - val_loss: 102.0699
Epoch 1787/10000
4/4 [==============================] - 0s 11ms/step - loss: 34.9914 - val_loss: 110.6277
Epoch 1788/10000
4/4 [==============================] - 0s 12ms/step - loss: 37.8926 - val_loss: 47.8027
Epoch 1789/10000
4/4 [==============================] - 0s 10ms/step - loss: 68.4762 - val_loss: 131.6597
Epoch 1790/10000
4/4 [==============================] - 0s 12ms/step - loss: 45.6231 - val_loss: 162.4577
Epoch 1791/10000
4/4 [==============================] - 0s 10ms/step - loss: 38.1890 - val_loss: 150.6663
Epoch 1792/10000
4/4 [==============================] - 0s 12ms/

4/4 [==============================] - 0s 12ms/step - loss: 36.2909 - val_loss: 138.8425
Epoch 1861/10000
4/4 [==============================] - 0s 11ms/step - loss: 39.8925 - val_loss: 116.4782
Epoch 1862/10000
4/4 [==============================] - 0s 12ms/step - loss: 48.4526 - val_loss: 120.0603
Epoch 1863/10000
4/4 [==============================] - 0s 13ms/step - loss: 40.2451 - val_loss: 112.8969
Epoch 1864/10000
4/4 [==============================] - 0s 13ms/step - loss: 39.7004 - val_loss: 100.1738
Epoch 1865/10000
4/4 [==============================] - 0s 12ms/step - loss: 32.6283 - val_loss: 137.9484
Epoch 1866/10000
4/4 [==============================] - 0s 11ms/step - loss: 40.6916 - val_loss: 138.7133
Epoch 1867/10000
4/4 [==============================] - 0s 12ms/step - loss: 30.5733 - val_loss: 139.3471
Epoch 1868/10000
4/4 [==============================] - 0s 12ms/step - loss: 29.4076 - val_loss: 144.8413
Epoch 1869/10000
4/4 [==============================] - 0s 12ms

4/4 [==============================] - 0s 12ms/step - loss: 33.6006 - val_loss: 169.5789
Epoch 1938/10000
4/4 [==============================] - 0s 11ms/step - loss: 40.7987 - val_loss: 147.7974
Epoch 1939/10000
4/4 [==============================] - 0s 11ms/step - loss: 29.4310 - val_loss: 155.4065
Epoch 1940/10000
4/4 [==============================] - 0s 11ms/step - loss: 28.6160 - val_loss: 155.0146
Epoch 1941/10000
4/4 [==============================] - 0s 12ms/step - loss: 29.4328 - val_loss: 156.5108
Epoch 1942/10000
4/4 [==============================] - 0s 12ms/step - loss: 29.4138 - val_loss: 143.8521
Epoch 1943/10000
4/4 [==============================] - 0s 10ms/step - loss: 34.0662 - val_loss: 144.5895
Epoch 1944/10000
4/4 [==============================] - 0s 12ms/step - loss: 35.2606 - val_loss: 144.0999
Epoch 1945/10000
4/4 [==============================] - 0s 12ms/step - loss: 29.4253 - val_loss: 139.8826
Epoch 1946/10000
4/4 [==============================] - 0s 11ms

4/4 [==============================] - 0s 12ms/step - loss: 27.8142 - val_loss: 134.8310
Epoch 2015/10000
4/4 [==============================] - 0s 11ms/step - loss: 28.5331 - val_loss: 144.8554
Epoch 2016/10000
4/4 [==============================] - 0s 12ms/step - loss: 43.2943 - val_loss: 137.0304
Epoch 2017/10000
4/4 [==============================] - 0s 12ms/step - loss: 35.4970 - val_loss: 158.6904
Epoch 2018/10000
4/4 [==============================] - 0s 11ms/step - loss: 28.0377 - val_loss: 156.8656
Epoch 2019/10000
4/4 [==============================] - 0s 11ms/step - loss: 28.5651 - val_loss: 141.6259
Epoch 2020/10000
4/4 [==============================] - 0s 11ms/step - loss: 31.8971 - val_loss: 148.3099
Epoch 2021/10000
4/4 [==============================] - 0s 10ms/step - loss: 31.7295 - val_loss: 140.5306
Epoch 2022/10000
4/4 [==============================] - 0s 12ms/step - loss: 27.6329 - val_loss: 137.5990
Epoch 2023/10000
4/4 [==============================] - 0s 12ms

4/4 [==============================] - 0s 12ms/step - loss: 24.6051 - val_loss: 120.6856
Epoch 2092/10000
4/4 [==============================] - 0s 12ms/step - loss: 23.4020 - val_loss: 119.6573
Epoch 2093/10000
4/4 [==============================] - 0s 12ms/step - loss: 21.9159 - val_loss: 144.3564
Epoch 2094/10000
4/4 [==============================] - 0s 12ms/step - loss: 31.6577 - val_loss: 148.8700
Epoch 2095/10000
4/4 [==============================] - 0s 12ms/step - loss: 30.7306 - val_loss: 177.8025
Epoch 2096/10000
4/4 [==============================] - 0s 12ms/step - loss: 28.6825 - val_loss: 130.5318
Epoch 2097/10000
4/4 [==============================] - 0s 12ms/step - loss: 27.6532 - val_loss: 138.5593
Epoch 2098/10000
4/4 [==============================] - 0s 12ms/step - loss: 22.0581 - val_loss: 129.3855
Epoch 2099/10000
4/4 [==============================] - 0s 12ms/step - loss: 23.8558 - val_loss: 125.0589
Epoch 2100/10000
4/4 [==============================] - 0s 12ms

In [53]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [54]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 16ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,103.622391,103.908348,104.217545,104.647011,104.901749,105.058563,105.213234,105.430069,105.604256,105.960251,106.356224,106.857727,106.647324,106.940765,107.035034,106.483261,106.292366,106.079178,105.792938,106.261841,105.813019,105.489128,106.348946,106.986603,107.968346,107.392258,107.327225,106.408012,106.081535,102.662476
Target,120.055,125.769,112.904,123.201,112.725,90.828,96.36,86.444,94.285,98.986,88.072,141.01,122.652,142.145,121.124,130.503,104.115,90.69,102.685,96.144,102.197,106.712,124.057,124.625,133.116,144.31,140.357,152.769,124.038,95.054
Error,16.43261,21.860649,8.686455,18.553986,7.82325,14.23056,8.853233,18.986069,11.319252,6.974251,18.284225,34.152267,16.004677,35.204239,14.088966,24.019745,2.177368,15.389175,3.107941,10.117844,3.61602,1.22287,17.708054,17.638397,25.147652,36.91774,33.02977,46.360985,17.956467,7.608475


In [55]:
display(mae)
display(mape)

17.115774

0.14200741

In [56]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [57]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

NameError: name 'test_target' is not defined